# Комментарий
Отлично. Всё правильно.  Успехов в итоговом проекте!

## Описание проекта
### Коронавирус застал мир врасплох, изменив привычный порядок вещей. В свободное время жители городов больше не выходят на улицу, не посещают кафе и торговые центры. Зато теперь они могут уделять больше времени книгам. На это обратили внимание стартаперы, которые бросились создавать новые приложения для тех, кто любит читать.

### Вам передали базу данных одного из конкурирующих сервисов. В ней есть информация о книгах, издательствах, авторах, а также пользовательские обзоры на книги. Эти данные помогут сформулировать ценностное предложение для нового продукта.

### Задания

- Посчитайте количество книг, выпущенных после 1 января 2000 года;
- Посчитайте количество пользовательских обзоров и среднюю оценку для каждой книги;
- Определите издательство, которое издало наибольшее число книг толще 50 страниц (так вы исключите из анализа различные брошюры);
- Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более пользовательскими оценками;
- Посчитайте среднее количество текстовых обзоров пользователей, которые поставили более чем по 50 оценок.

In [1]:
import pandas as pd
from sqlalchemy import create_engine
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
db_config['pwd'],
db_config['host'],
db_config['port'],
db_config['db'])
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

In [2]:
def select(sql):
    return pd.io.sql.read_sql(sql, con = engine)

In [3]:
query = '''
SELECT * FROM books
'''
books = pd.io.sql.read_sql(query, con = engine)
query_1 = '''
SELECT * FROM authors
'''

authors = pd.io.sql.read_sql(query_1, con = engine)
query_2 = '''
SELECT * FROM publishers
'''

publishers = pd.io.sql.read_sql(query_2, con = engine)
query_3 = '''
SELECT * FROM ratings
'''

ratings = pd.io.sql.read_sql(query_3, con = engine)
query_4 = '''
SELECT * FROM reviews
'''

reviews = pd.io.sql.read_sql(query_4, con = engine)

### Посчитайте количество книг, выпущенных после 1 января 2000 года;

In [8]:
books_cnt = '''
SELECT COUNT(book_id) AS books_cnt FROM books WHERE publication_date >= '2000-01-02'
'''
select(books_cnt)

,books_cnt
0,819


После 1 января 2000 года было выпущено 819 книг

### Посчитайте количество пользовательских обзоров и среднюю оценку для каждой книги;

In [15]:
ratings_reviews = '''
SELECT 
    books.title AS title,
    books.book_id AS id,
    AVG(ratings.rating) AS avg_rating,
    COUNT(DISTINCT reviews.text) AS cnt_reviews
FROM
    books
INNER JOIN ratings ON ratings.book_id = books.book_id
INNER JOIN reviews ON reviews.book_id = books.book_id
GROUP BY
     books.title,
     books.book_id
order by
     cnt_reviews DESC
limit 20
'''
select(ratings_reviews)

,title,id,avg_rating,cnt_reviews
0,Twilight (Twilight #1),948,3.662500,7
1,The Alchemist,627,3.789474,6
2,Outlander (Outlander #1),497,4.125000,6
3,The Hobbit or There and Back Again,750,4.125000,6
4,Eat Pray Love,207,3.395833,6
5,The Book Thief,656,4.264151,6
6,The Curious Incident of the Dog in the Night-Time,695,4.081081,6
7,The Catcher in the Rye,673,3.825581,6
8,The Giver (The Giver #1),733,3.750000,6
9,The Glass Castle,734,4.206897,6


Для 7 книг отсуствуют рейтинг и обзор, максимальное кол-во обзоров у книги Twilight (Twilight #1). Достаточно много книг с оценкой 5, но у них всего 2 обзора.

### Определите издательство, которое издало наибольшее число книг толще 50 страниц (так вы исключите из анализа различные брошюры);

In [16]:
book_publisher = '''
SELECT
     publishers.publisher AS name_publisher,
     COUNT(books.book_id) AS cnt_of_published_books
FROM
    publishers
INNER JOIN books ON books.publisher_id = publishers.publisher_id
WHERE
    books.num_pages > 50
GROUP BY
    name_publisher
ORDER BY 
    cnt_of_published_books DESC
LIMIT 20
'''
select(book_publisher)

,name_publisher,cnt_of_published_books
0,Penguin Books,42
1,Vintage,31
2,Grand Central Publishing,25
3,Penguin Classics,24
4,Bantam,19
5,Ballantine Books,19
6,Berkley,17
7,Berkley Books,14
8,St. Martin's Press,14
9,William Morrow Paperbacks,13


 Издательство Penguin Books - выпустило наибольше число книг в кол-ве 42 книг, толще 50 страниц.

### Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более пользовательскими оценками;

In [14]:
best_author_ever = '''
SELECT
      SUBQ.author AS author,
      AVG(SUBQ.avg_rating) AS avg_rating
FROM
(SELECT
     authors.author AS author,
     books.book_id AS id,
     AVG(ratings.rating) AS avg_rating,
     COUNT(ratings.rating) AS cnt_rating
FROM
    authors
INNER JOIN books ON books.author_id = authors.author_id
INNER JOIN ratings ON ratings.book_id = books.book_id
GROUP BY
    author,
    id
    HAVING
    COUNT(ratings.rating) >= 50) AS SUBQ
GROUP BY
    author
ORDER BY
    avg_rating DESC
'''
select(best_author_ever)

,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.283844
1,Markus Zusak/Cao Xuân Việt Khương,4.264151
2,J.R.R. Tolkien,4.258446
3,Louisa May Alcott,4.192308
4,Rick Riordan,4.080645
5,William Golding,3.901408
6,J.D. Salinger,3.825581
7,Paulo Coelho/Alan R. Clarke/Özdemir İnce,3.789474
8,William Shakespeare/Paul Werstine/Barbara A. M...,3.787879
9,Dan Brown,3.754540


Cамая высокая средняя оценка у автора J.K. Rowling/Mary GrandPré

### Посчитайте среднее количество текстовых обзоров пользователей, которые поставили более чем по 50 оценок

In [7]:
avg_text = '''
SELECT ROUND(avg(count)) 
FROM
(SELECT
    COUNT(text)
FROM 
    reviews
FULL OUTER JOIN (SELECT username,
                COUNT(rating_id) as cnt
FROM 
    ratings
GROUP BY
        username) AS cnt_rating
ON cnt_rating.username = reviews.username
where cnt > 50
GROUP BY reviews.username) as count;                                  
'''
select(avg_text)

,round
0,24.0


Среднее кол-во текстовых обзоров пользователей, которые поставили более чем по 50 оценок равно 24

### Общий вывод:
- В таблице book_publisher лежит список всех издательств, которые выпускают книги от 50 страниц
- Таблица best_author_ever, в ней лежит список авторов с самой выской средней оценкой, так как у этих авторов учтены книги которые имеют больше 50 пользовательских оценок, то эти оценки можно считать "правдивыми". Обязательно нужно включить книги этих авторов, а в таблице ratings_reviews, как раз есть книги со средней оценкой и пользовательскими обзорами, стоит обратить внимание на книги, где кол-во обзоров больше 3, так как достаточно много завышенных оценок с 2 обзорами . Так же среднее количество текстовых обзоров на книги более чем 50 оценками имеют в среднем 24 текстовых обзора, что будет достаточно информативно для будущих пользователей.
